- DistilBERT를 이용한 감성분석 모델
- KoBERT 모델과 모든 테스크 동일
-  

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import sys

print(os.getcwd())

/content


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.5 MB/s eta 0:00:00
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_c

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/dndhma/KoBERT.git@master

  Cloning https://****@github.com/dndhma/KoBERT.git (to revision master) to /tmp/pip-req-build-fcfbh45l
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/dndhma/KoBERT.git' /tmp/pip-req-build-fcfbh45l
  Resolved https://****@github.com/dndhma/KoBERT.git to commit ba3cfd3dd7b0781d588bb571b25a4ac1bc88f2ac
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.15.18-py2.py3-none-any.whl (129 kB)
  Using cached mxnet-1.7.0.post2-py2.py3-none-manylinux2014_x86_64.whl (54.7 MB)
  Using cached onnxruntime-1.18.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.8 MB)
  Using cached sentencepiece-0.1.96-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached botocore-1.18.18-py2.py3-none-any.whl (6.7 MB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached s3tr

# Data Preparation and Training

In [5]:
!pip install tokenizers
import numpy as np
np.bool = np.bool_
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

In [6]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
#GPU 사용
device = torch.device("cuda")

In [8]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [9]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [13]:
import pandas as pd
chatbot_data = pd.read_csv('sentiment_dialogues.csv')

In [16]:
chatbot_data.loc[(chatbot_data[ '감정'] == "불안"),  '감정'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data[ '감정'] == "당황"),  '감정'] = 1  #당황 => 1
chatbot_data.loc[(chatbot_data[ '감정'] == "분노"),  '감정'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data[ '감정'] == "슬픔"),  '감정'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data[ '감정'] == "중립"),  '감정'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data[ '감정'] == "행복"),  '감정'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data[ '감정'] == "혐오"),  '감정'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['발화'], chatbot_data[ '감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [17]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.13, random_state=0)

In [18]:
print(len(dataset_train))
print(len(dataset_test))

127513
19054


In [19]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [20]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [21]:
# 토큰화
tokenizer = get_tokenizer()

# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [23]:
#kobert 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        pooler = self.bert(input_ids = token_ids, attention_mask = attention_mask.float().to(token_ids.device))[0][:,0]
        # distilbert는 pooler를 return하지 않음. 따라서 last_hidden_state로 부터 직접 pooler를 추출함

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
#BERT 모델 불러오기
model = BERTClassifier(distilbert_model,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
best_acc=0.0
best_loss=99999999
ckpt_path="/content/drive/MyDrive/" #your path
ckpt_name=ckpt_path+"saved_model_distilbert.pt"

In [ ]:
#kobert 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long().to(device) ## DistillBERT는 token_type_ids(segment_ids)를 사용하지 않음.
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        with torch.no_grad():
          out = model(token_ids, valid_length)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    if test_acc>best_acc and test_loss.data.cpu().numpy()<best_loss:
      torch.save({'epoch':e+1,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'loss':test_loss.data.cpu().numpy()},
                 ckpt_name)
      best_loss=test_loss.data.cpu().numpy()
      bset_acc=test_acc

      print('current best model saved')


<ipython-input-31-0b7efc5f2966>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1993 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.8964858055114746 train acc 0.65625


# Inference

In [ ]:
#새로운 문장 테스트
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()
    with torch.no_grad():

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)


          test_eval=[]
          for i in out:
              logits=i
              logits = logits.detach().cpu().numpy()

              if np.argmax(logits) == 0:
                  return "불안"
              elif np.argmax(logits) == 1:
                  return "당황이"
              elif np.argmax(logits) == 2:
                  return "분노가"
              elif np.argmax(logits) == 3:
                  return "슬픔이"
              elif np.argmax(logits) == 4:
                  return "중립이"
              elif np.argmax(logits) == 5:
                  return "행복이"
              elif np.argmax(logits) == 6:
                  return "혐오가"


In [ ]:
checkpoint=torch.load(ckpt_name)

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
import json

"""
format: [{"title":"[💙자유]단대소고 설명회 갔는데요...","content":"강당 자리를 사람들이 거의 다 채워서 1번 놀라고\n그 앞에서 에어컨 자랑, 급식 자랑하시는 교장님을 보고 2번 놀랐습니다\n교장님 귀여우시네요\n","comments":["어우 쒯 ㅋㅋ","학교 자랑한답시고 하는게 “에어컨”, “급식” ㅋㅋㅋㅋㅋㅋ","급식으로 매일 짜장면 시켜드시는  교직원분도 계신디...쓰읍","???:단대부고 화이팅","conalt=\"킹받네\""]},
"""

res = {}
# save data in format of


json_data = open('output.json', 'r', encoding='utf-8').read()
# Parse JSON data
data = json.loads(json_data)

# Define the predict function
def predict(text):
    # Placeholder for the actual prediction logic
    print(f"Predicting for: {text}")

# Iterate over all content and comments
for entry in data:
    # Process content
    if entry['content'] == '':
        continue
    res[entry['content']] = predict(entry['content'])

    # Process comments
    for comment in entry['comments']:
        if comment == '':
            continue
        res[comment] = predict(comment)

## save
with open('processed.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)